## Add imports

In [76]:
import sys, platform, importlib
from pathlib import Path
import numpy as np
from PIL import Image
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import joblib
import skimage
from skimage import color, transform
from skimage.feature import hog

# Use parent-relative paths as requested
DATA_TRAIN = Path('../data/train')
DATA_TEST = Path('../data/test')
MODELS_DIR = Path('models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_DIR = Path('outputs')
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_STATE = 123
IMAGE_SIZE = (224, 224)  # (H, W) default

## prepare dataset

In [77]:
from pathlib import Path as _Path
CLASS_MAP = {'AP': 0, 'Lateral': 1}
IMG_EXTS = ('.png')

def collect_paths(root: Path) -> list[tuple[Path, int]]:
    """Return list of (image_path, label) pairs from a root folder.
    Expects subfolders named after CLASS_MAP keys, e.g., 'AP' and 'Lateral'.
    """
    items: list[tuple[Path, int]] = []
    for cls_name, lbl in CLASS_MAP.items():
        cls_dir = root / cls_name
        if not cls_dir.exists():
            print(f"Warning: missing folder: {cls_dir}")
            continue
        # Gather all supported image files
        file_list: list[Path] = []
        for ext in IMG_EXTS:
            file_list.extend(sorted(cls_dir.rglob(f"*{ext}")))
        # Append with labels
        for p in file_list:
            items.append((p, lbl))
    # Stable ordering by (label, path)
    items.sort(key=lambda t: (t[1], str(t[0]).lower()))
    return items

train_items = collect_paths(DATA_TRAIN)
print(f"Training images: {len(train_items)} from {DATA_TRAIN}")

Training images: 184 from ..\data\train


## feature extraction

In [78]:
from typing import Tuple

def compute_hog_from_pil(img: Image, image_size: Tuple[int,int], pixels_per_cell: Tuple[int,int], cells_per_block: Tuple[int,int], orientations: int) -> np.ndarray:
    # Resize and grayscale
    img_np = np.array(img)
    if img_np.ndim == 3:
        gray = color.rgb2gray(img_np)
    else:
        gray = img_np.astype(np.float32)
        rng = gray.max() - gray.min()
        gray = (gray - gray.min()) / (rng if rng > 1e-6 else 1.0)
    gray_resized = transform.resize(gray, image_size, anti_aliasing=True)

    feat = hog(
        gray_resized,
        pixels_per_cell=pixels_per_cell,
        cells_per_block=cells_per_block,
        orientations=orientations,
        block_norm='L2-Hys',
        feature_vector=True,
    )
    return feat.astype(np.float32)

In [79]:
PPC_GRID = [(16, 16), (8, 8)]
CPB_GRID = [(2, 2), (3, 3)]
ORI_GRID = [9, 12]
C_GRID = [0.1, 1.0, 10.0]
print('Grids set:', {'ppc': PPC_GRID, 'cpb': CPB_GRID, 'ori': ORI_GRID, 'C': C_GRID})

Grids set: {'ppc': [(16, 16), (8, 8)], 'cpb': [(2, 2), (3, 3)], 'ori': [9, 12], 'C': [0.1, 1.0, 10.0]}


## training setup

In [80]:
y_all = np.array([lbl for _, lbl in train_items], dtype=np.int64)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
print('Label distribution:', {int(c): int(np.sum(y_all==c)) for c in np.unique(y_all)})

Label distribution: {0: 104, 1: 80}


In [81]:
results = []
best = {'mean_acc': -1.0, 'std_acc': 0.0, 'ppc': None, 'cpb': None, 'ori': None, 'C': None, 'loss': 'squared_hinge', 'dual': True, 'n_features': None}

def _compute_all_features(items, ppc, cpb, ori):
    feats = []
    for pth, _label in items:
        img = Image.open(pth).convert('RGB')
        f = compute_hog_from_pil(img, IMAGE_SIZE, tuple(ppc), tuple(cpb), int(ori))
        feats.append(f)
    X_all = np.stack(feats)
    return X_all

# Iterate HOG grids and precompute features
grid_feature_cache = {}
for ppc in PPC_GRID:
    for cpb in CPB_GRID:
        for ori in ORI_GRID:
            X_all = _compute_all_features(train_items, ppc, cpb, ori)
            grid_feature_cache[(ppc, cpb, ori)] = X_all
            print(f'Computed features size={IMAGE_SIZE} ppc={ppc} cpb={cpb} ori={ori} -> feat_dim={X_all.shape[1]}')

Computed features size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 -> feat_dim=6084
Computed features size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 -> feat_dim=8112
Computed features size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 -> feat_dim=8112
Computed features size=(224, 224) ppc=(16, 16) cpb=(3, 3) ori=9 -> feat_dim=11664
Computed features size=(224, 224) ppc=(16, 16) cpb=(3, 3) ori=9 -> feat_dim=11664
Computed features size=(224, 224) ppc=(16, 16) cpb=(3, 3) ori=12 -> feat_dim=15552
Computed features size=(224, 224) ppc=(16, 16) cpb=(3, 3) ori=12 -> feat_dim=15552
Computed features size=(224, 224) ppc=(8, 8) cpb=(2, 2) ori=9 -> feat_dim=26244
Computed features size=(224, 224) ppc=(8, 8) cpb=(2, 2) ori=9 -> feat_dim=26244
Computed features size=(224, 224) ppc=(8, 8) cpb=(2, 2) ori=12 -> feat_dim=34992
Computed features size=(224, 224) ppc=(8, 8) cpb=(2, 2) ori=12 -> feat_dim=34992
Computed features size=(224, 224) ppc=(8, 8) cpb=(3, 3) ori=9 -> feat_dim=54756
Computed features size=

In [82]:
for ppc in PPC_GRID:
    for cpb in CPB_GRID:
        for ori in ORI_GRID:
            X_all = grid_feature_cache[(ppc, cpb, ori)]
            for C in C_GRID:
                fold_accs = []
                for tr_idx, va_idx in skf.split(X_all, y_all):
                    Xtr, Xva = X_all[tr_idx], X_all[va_idx]
                    ytr, yva = y_all[tr_idx], y_all[va_idx]
                    clf = Pipeline([
                        ('scaler', StandardScaler(with_mean=True)),
                        ('svm', LinearSVC(C=float(C), loss='squared_hinge', dual=True, class_weight='balanced', max_iter=5000)),
                    ])
                    clf.fit(Xtr, ytr)
                    yhat = clf.predict(Xva)
                    fold_accs.append(float(accuracy_score(yva, yhat)))
                mean_acc = float(np.mean(fold_accs))
                std_acc = float(np.std(fold_accs))
                res = {
                    'pixels_per_cell': tuple(ppc),
                    'cells_per_block': tuple(cpb),
                    'orientations': int(ori),
                    'C': float(C),
                    'loss': 'squared_hinge',
                    'dual': True,
                    'mean_acc': mean_acc,
                    'std_acc': std_acc,
                    'n_features': int(X_all.shape[1]),
                }
                results.append(res)
                print(f"CV size={IMAGE_SIZE} ppc={ppc} cpb={cpb} ori={ori} | C={C} -> acc {mean_acc:.3f} ± {std_acc:.3f} (feat={X_all.shape[1]})")
                # Tie-breaking: higher mean_acc, then lower n_features, then lower C
                if (mean_acc > best['mean_acc']) or (
                    np.isclose(mean_acc, best['mean_acc']) and X_all.shape[1] < (best['n_features'] or 1e12)
                ) or (
                    np.isclose(mean_acc, best['mean_acc']) and X_all.shape[1] == (best['n_features'] or 1e12) and float(C) < float(best['C'] or 1e12)
                ):
                    best.update({
                        'mean_acc': mean_acc,
                        'std_acc': std_acc,
                        'ppc': tuple(ppc),
                        'cpb': tuple(cpb),
                        'ori': int(ori),
                        'C': float(C),
                        'loss': 'squared_hinge',
                        'dual': True,
                        'n_features': int(X_all.shape[1]),
                    })

CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 | C=0.1 -> acc 0.951 ± 0.020 (feat=6084)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 | C=1.0 -> acc 0.951 ± 0.020 (feat=6084)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 | C=1.0 -> acc 0.951 ± 0.020 (feat=6084)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 | C=10.0 -> acc 0.951 ± 0.020 (feat=6084)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=9 | C=10.0 -> acc 0.951 ± 0.020 (feat=6084)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=0.1 -> acc 0.935 ± 0.014 (feat=8112)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=0.1 -> acc 0.935 ± 0.014 (feat=8112)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=1.0 -> acc 0.935 ± 0.014 (feat=8112)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=1.0 -> acc 0.935 ± 0.014 (feat=8112)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=10.0 -> acc 0.935 ± 0.014 (feat=8112)
CV size=(224, 224) ppc=(16, 16) cpb=(2, 2) ori=12 | C=10.0 -> acc 0.935 ± 0.014 (feat=8112)


In [83]:
if best['ppc'] is None:
    # Fallback to first grid entries
    best['ppc'] = PPC_GRID[0]
    best['cpb'] = CPB_GRID[0]
    best['ori'] = int(ORI_GRID[0])
print('Best so far:', best)

Best so far: {'mean_acc': 0.9566066066066066, 'std_acc': 0.02151081047332879, 'ppc': (16, 16), 'cpb': (3, 3), 'ori': 9, 'C': 0.1, 'loss': 'squared_hinge', 'dual': True, 'n_features': 11664}


## model training

In [84]:
X_best, y_best = [], []
for pth, label in train_items:
    img = Image.open(pth).convert('RGB')
    f = compute_hog_from_pil(img, IMAGE_SIZE, tuple(best['ppc']), tuple(best['cpb']), int(best['ori']))
    X_best.append(f)
    y_best.append(label)
X_best = np.stack(X_best)
y_best = np.array(y_best, dtype=np.int64)

final_clf = Pipeline([
    ('scaler', StandardScaler(with_mean=True)),
    ('svm', LinearSVC(C=float(best.get('C', 1.0)), loss='squared_hinge', dual=True, class_weight='balanced', max_iter=5000)),
])
final_clf.fit(X_best, y_best)

,steps,"[('scaler', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,penalty,'l2'
,loss,'squared_hinge'
,dual,True
,tol,0.0001


In [85]:
yhat = final_clf.predict(X_best)
acc = accuracy_score(y_best, yhat)
print('Best params:')
print(f"  pixels_per_cell={tuple(best['ppc'])}  cells_per_block={tuple(best['cpb'])}  orientations={best['ori']}")
print(f"  SVM: C={best.get('C', 1.0)}  loss={'squared_hinge'}  dual={True}")
print(f"  CV mean acc={best['mean_acc']:.3f} ± {best['std_acc']:.3f} | resub acc={acc:.3f}")

Best params:
  pixels_per_cell=(16, 16)  cells_per_block=(3, 3)  orientations=9
  SVM: C=0.1  loss=squared_hinge  dual=True
  CV mean acc=0.957 ± 0.022 | resub acc=1.000


In [86]:
artifact = {
    'pipeline': final_clf,
    'hog_params': {
        'image_size': IMAGE_SIZE,
        'pixels_per_cell': tuple(best['ppc']),
        'cells_per_block': tuple(best['cpb']),
        'orientations': int(best['ori']),
    },
    'n_features': int(X_best.shape[1]),
    'n_samples': int(X_best.shape[0]),
    'cv_results': results,
    'cv_folds': int(skf.n_splits),
    'random_state': int(RANDOM_STATE),
    'cv_best': {
        'mean_acc': float(best['mean_acc']),
        'std_acc': float(best['std_acc']),
    },
    'svm_params': {
        'C': float(best.get('C', 1.0)),
        'loss': 'squared_hinge',
        'dual': True,
    },
}
out_path = MODELS_DIR / 'hog_linear_svm.joblib'
joblib.dump(artifact, out_path)
print(f'Saved model artifact (with CV) to {out_path}')

Saved model artifact (with CV) to models\hog_linear_svm.joblib


## Inference

In [87]:
import warnings as _warnings

SVM_PATH = MODELS_DIR / 'hog_linear_svm.joblib'
SVC_PATH = MODELS_DIR / 'hog_linear_svc.joblib'
to_load = None
if SVM_PATH.exists():
    to_load = SVM_PATH
elif SVC_PATH.exists():
    to_load = SVC_PATH
else:
    print('No model found. Train first (cells 1-11).')

EVAL_IMAGE_SIZE = IMAGE_SIZE
EVAL_PPC = (16, 16)
EVAL_CPB = (2, 2)
EVAL_ORI = 9
reload_clf = None

if to_load:
    loaded = joblib.load(to_load)
    if isinstance(loaded, dict):
        reload_clf = loaded.get('pipeline')
        hp = loaded.get('hog_params', {})
        # Prefer embedded hog params if present
        EVAL_IMAGE_SIZE = tuple(hp.get('image_size', IMAGE_SIZE))
        EVAL_PPC = tuple(hp.get('pixels_per_cell', EVAL_PPC))
        EVAL_CPB = tuple(hp.get('cells_per_block', EVAL_CPB))
        EVAL_ORI = int(hp.get('orientations', EVAL_ORI))
        print(f"Using artifact hog_params: size={EVAL_IMAGE_SIZE} ppc={EVAL_PPC} cpb={EVAL_CPB} ori={EVAL_ORI}")
    else:
        reload_clf = loaded
        _warnings.warn('Artifact missing hog_params; using script defaults for evaluation.')
        print(f"Using defaults: size={EVAL_IMAGE_SIZE} ppc={EVAL_PPC} cpb={EVAL_CPB} ori={EVAL_ORI}")

assert reload_clf is not None, 'Failed to load a classifier pipeline.'

Using artifact hog_params: size=(224, 224) ppc=(16, 16) cpb=(3, 3) ori=9


## save misclassified examples

In [88]:
import matplotlib.pyplot as plt
from math import ceil

test_items = collect_paths(DATA_TEST)
print(f'Test images: {len(test_items)} from {DATA_TEST}')

if len(test_items) == 0:
    print('No test images found. Create data/test/{AP,Lateral} folders.')
else:
    # Build feature matrix with EVAL_* params
    X_new, y_new, paths_new = [], [], []
    for p, label in test_items:
        img = Image.open(p).convert('RGB')
        feat = compute_hog_from_pil(
            img,
            image_size=EVAL_IMAGE_SIZE,
            pixels_per_cell=EVAL_PPC,
            cells_per_block=EVAL_CPB,
            orientations=EVAL_ORI,
        )
        X_new.append(feat)
        y_new.append(label)
        paths_new.append(p)
    X_new = np.stack(X_new)
    y_new = np.array(y_new, dtype=np.int64)

    preds = reload_clf.predict(X_new)
    acc_new = accuracy_score(y_new, preds)
    print(f'Test accuracy: {acc_new:.3f}')

    # Identify misclassifications
    mis_idx = [i for i in range(len(preds)) if preds[i] != y_new[i]]
    print(f'Misclassified: {len(mis_idx)} of {len(preds)}')

    # Save grid of failed samples
    if mis_idx:
        max_save = 60
        grid_cols = 6
        sel = mis_idx[:max_save]
        n = len(sel)
        rows = ceil(n / grid_cols) if n > 0 else 1
        fig, axes = plt.subplots(rows, grid_cols, figsize=(grid_cols*2.2, rows*2.2))
        axes = np.array(axes).reshape(rows, grid_cols)
        for k in range(rows*grid_cols):
            r, c = divmod(k, grid_cols)
            ax = axes[r, c]
            ax.axis('off')
            if k < n:
                i = sel[k]
                img = Image.open(paths_new[i]).convert('RGB')
                ax.imshow(img)
                ax.set_title(f"T={y_new[i]} P={preds[i]}", fontsize=8)
        out_dir = Path('outputs') / 'hog_errors'
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / 'failed_grid.png'
        plt.tight_layout()
        plt.savefig(out_path, dpi=150)
        plt.close(fig)
        print(f'Save misclassified images: {out_path}')
    else:
        print('No misclassifications to save.')

Test images: 27 from ..\data\test
Test accuracy: 0.852
Misclassified: 4 of 27
Test accuracy: 0.852
Misclassified: 4 of 27
Save misclassified images: outputs\hog_errors\failed_grid.png
Save misclassified images: outputs\hog_errors\failed_grid.png
